In [10]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
from tensorflow.keras import Model, mixed_precision

import os

os.chdir('..')

In [11]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [30]:
from sam import sam_train_step

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, dtype='float32')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

class MyModelSAM(Model):
  def __init__(self):
    super(MyModelSAM, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, dtype='float32')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
  
  def train_step(self, data):
    return sam_train_step(self, data)

# Create an instance of the model
model = MyModel()
model.build((None, 28, 28, 1))
sam_model = MyModelSAM()
sam_model.build((None, 28, 28, 1))
sam_model.set_weights(model.get_weights())

In [31]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_ds,
          epochs=5,
          validation_data=test_ds)

Epoch 1/5
1875/1875 [==============================] - 61s 32ms/step - loss: 0.1343 - accuracy: 0.9597 - val_loss: 0.0720 - val_accuracy: 0.9759
Epoch 2/5
1875/1875 [==============================] - 59s 32ms/step - loss: 0.0502 - accuracy: 0.9853 - val_loss: 0.0495 - val_accuracy: 0.9839
Epoch 3/5
1875/1875 [==============================] - 59s 32ms/step - loss: 0.0331 - accuracy: 0.9903 - val_loss: 0.0513 - val_accuracy: 0.9853
Epoch 4/5
1875/1875 [==============================] - 60s 32ms/step - loss: 0.0237 - accuracy: 0.9934 - val_loss: 0.0513 - val_accuracy: 0.9857
Epoch 5/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.0168 - accuracy: 0.9957 - val_loss: 0.0555 - val_accuracy: 0.9854


In [ ]:
sam_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
sam_model.fit(train_ds,
              epochs=5,
              validation_data=test_ds)

Epoch 1/5
1875/1875 [==============================] - 91s 48ms/step - loss: 0.1536 - accuracy: 0.9467 - val_loss: 0.0607 - val_accuracy: 0.9801
Epoch 2/5
1875/1875 [==============================] - 95s 51ms/step - loss: 0.0631 - accuracy: 0.9761 - val_loss: 0.0483 - val_accuracy: 0.9841
Epoch 3/5
1875/1875 [==============================] - 90s 48ms/step - loss: 0.0433 - accuracy: 0.9833 - val_loss: 0.0500 - val_accuracy: 0.9853
Epoch 4/5
1875/1875 [==============================] - 92s 49ms/step - loss: 0.0306 - accuracy: 0.9875 - val_loss: 0.0578 - val_accuracy: 0.9842
Epoch 5/5
 375/1875 [=====>........................] - ETA: 1:10 - loss: 0.0219 - accuracy: 0.9908